In [83]:
!python ../scripts/training/train.py --config ./config.yaml

2025-01-01 03:19:52,255 - /home/arda/Documents/chronos-forecasting/notebooks/../scripts/training/train.py - INFO - Using SEED: 1497243461
2025-01-01 03:19:52,260 - /home/arda/Documents/chronos-forecasting/notebooks/../scripts/training/train.py - INFO - Setting the number of data loader workers to 2, instead of 8.
2025-01-01 03:19:52,261 - /home/arda/Documents/chronos-forecasting/notebooks/../scripts/training/train.py - INFO - Logging dir: output/run-16
2025-01-01 03:19:52,261 - /home/arda/Documents/chronos-forecasting/notebooks/../scripts/training/train.py - INFO - Loading and filtering 2 datasets for training: ['../data/tsmixup-data.arrow', '../data/kernelsynth-data.arrow']
2025-01-01 03:19:52,261 - /home/arda/Documents/chronos-forecasting/notebooks/../scripts/training/train.py - INFO - Mixing probabilities: [0.9, 0.1]
2025-01-01 03:19:52,263 - /home/arda/Documents/chronos-forecasting/notebooks/../scripts/training/train.py - INFO - Initializing model
2025-01-01 03:19:52,263 - /home/ar